<a href="https://colab.research.google.com/github/entangelk/study_AIs/blob/main/docs/quests/NLPs/classification_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install wget

In [ ]:
import wget

In [ ]:
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

'ratings_train (2).txt'

In [ ]:
import pandas as pd

/tmp/ipykernel_21/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
get_data = pd.read_csv('ratings_train.txt', sep='\t')
pd.read_csv('ratings_train.txt', sep='\t')[:3]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [ ]:
# !apt-get -y install fonts-nanum
# !fc-cache -fv
# !rm ~/.cache/matplotlib -rf

## 데이터 전처리

In [ ]:
get_data.isna().sum()

id          0
document    5
label       0
dtype: int64

In [ ]:
get_data=get_data.dropna()

In [ ]:
import re

clear_list = []
for i in get_data['document']:
    get_text = re.sub(r'\.{2,}', ' ', i)
    get_text = get_text.lower()
    clear_list.append(get_text)

- 마침표, 말줄임표는 부정 긍정에 모두 포함되어 있으므로 삭제
- 숫자는 연령대에 대한 내용, 연도에 대한 감상이 담길 수 있으므로 살림
- 영어는 소문자로 모두 처리하여 진행

In [ ]:
clear_list[:5]

['아 더빙  진짜 짜증나네요 목소리',
 '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼  솔직히 재미는 없다 평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']

In [ ]:
!pip install python-mecab-ko

In [ ]:
from mecab import MeCab
mecab = MeCab()

In [ ]:
mecab.pos(clear_list[5])

[('막', 'MAJ'),
 ('걸음마', 'NNG'),
 ('뗀', 'VV+ETM'),
 ('3', 'SN'),
 ('세', 'NNBC'),
 ('부터', 'JX'),
 ('초등', 'NNG'),
 ('학교', 'NNG'),
 ('1', 'SN'),
 ('학년', 'NNG'),
 ('생', 'XSN'),
 ('인', 'VCP+ETM'),
 ('8', 'SN'),
 ('살', 'NNBC'),
 ('용', 'XSN'),
 ('영화', 'NNG'),
 ('.', 'SF'),
 ('ㅋㅋㅋ', 'IC'),
 ('별반', 'MAG'),
 ('개', 'NNG'),
 ('도', 'JX'),
 ('아까움', 'VA+ETN'),
 ('.', 'SF')]

In [ ]:
def mecabing (get_list):
    noun_sym_list=[]
    for i in get_list:
        morphas = mecab.pos(i)
        list_text = ''
        for k,v in morphas:
            if v == 'MAG' or v == 'NNG' or v =='SY' or v =='VA+ETN' or v=='SN':
                list_text += k
                list_text += ' '
            if k in r'[ㄱ-ㅣ]':
                list_text += k
                list_text += ' '
        noun_sym_list.append(list_text)
    return noun_sym_list

In [ ]:
noun_sym_list = mecabing(clear_list)
noun_sym_list[:3]

['더 빙 진짜 짜증 목소리 ', '초딩 영화 줄 오버 연기 ', '너무 ']

In [ ]:
get_data['doc_clear'] = noun_sym_list
get_data[:5]

,id,document,label,doc_clear
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,더 빙 진짜 짜증 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,초딩 영화 줄 오버 연기
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 솔직히 재미 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,익살 연기 영화 ! 너무나


## 벡터화

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None)

In [ ]:
tfidfVectorizer.fit(get_data['doc_clear'])

TfidfVectorizer()

## 트레인 셋, 테스트셋 분리

In [ ]:
train_set = get_data['doc_clear']
train_target = get_data['label']

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(train_set, train_target, test_size=0.3, random_state=55)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((104996,), (44999,), (104996,), (44999,))

In [ ]:
features_train[:3]

128349                                               너무 너무 
148692                                               형제 영화 
6520      가족 그리움 별 인간미 인간 세계 얼마나 모순 가득 한지 얘기 시선 줄곧 제공 친절...
Name: doc_clear, dtype: object

In [ ]:
features_test[:3]

94829     90 60 대풍 싸구려 삼류 모방작 . 3 . 8 20 발정 불알 8 . 78 땡 ...
85071                                                스토리 좀 
105159                                 대박 다 눈물 기류 영화 진짜 대박 
Name: doc_clear, dtype: object

## 모델

In [ ]:
train_vector_set = tfidfVectorizer.transform(features_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model

DecisionTreeClassifier()

In [ ]:
model.fit(train_vector_set,target_train)

DecisionTreeClassifier()

## 평가

In [ ]:
sample_nag = "와 이 영화 진짜 최악이에요. 바보가 만들어도 이것보단 잘만들듯 ㅋㅋㅋ"

In [ ]:
sample_nag_mecab = mecabing([sample_nag])

In [ ]:
sample_nag_mecab

['영화 진짜 최악 바보 잘 ']

In [ ]:
model.predict(tfidfVectorizer.transform(sample_nag_mecab))

array([0])

In [ ]:
sample_pos = "너무 감동적이에요. 진짜 재미있었어요!"

In [ ]:
sample_pos_mecab = mecabing([sample_pos])

In [ ]:
sample_pos_mecab

['너무 감동 진짜 ']

In [ ]:
model.predict(tfidfVectorizer.transform(sample_pos_mecab))

array([1])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
target_predict = model.predict(tfidfVectorizer.transform(features_test))
print(classification_report(target_test, target_predict))

              precision    recall  f1-score   support

           0       0.70      0.63      0.66     22563
           1       0.66      0.73      0.69     22436

    accuracy                           0.68     44999
   macro avg       0.68      0.68      0.68     44999
weighted avg       0.68      0.68      0.68     44999



## 모델 평가결과

f1 score간 차이가 적고 가중평균이 높아 모델의 정확도에 대한 신뢰성은 높은 편이나 f1 socre와 정확도가 0.7을 하회하여 정확도에 대한 처리를 더욱더 하면 좋을 것 같다.